In [1354]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin, BaseEstimator, clone
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer, RobustScaler
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow import keras
plt.rcParams["figure.figsize"] = (20,15)

In [1355]:
## Loading data obtained after data cleaning

def load_final_data():
    x_train = pd.read_csv("x_train_final.csv")
    x_val = pd.read_csv("x_val_final.csv")
    y_train = pd.read_csv('y_train_final.csv')
    y_val = pd.read_csv('y_val_final.csv')
    return x_train, y_train['fare_amount'], x_val, y_val['fare_amount']

In [1356]:
x_train, y_train, x_val, y_val = load_final_data()

In [1357]:
# impute the Nan values
# mode imputing categorical variables and median imputing the numerical 

In [1358]:
x_train

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,congestion_surcharge,tpep_pickup_datetime,tpep_dropoff_datetime
0,2.0,1.0,7.39,1.0,N,113,129,1.0,2.5,2020-06-11 21:10:27,2020-06-11 21:39:42
1,2.0,1.0,3.12,1.0,N,140,224,1.0,2.5,2020-06-23 10:28:28,2020-06-23 10:43:48
2,1.0,2.0,18.80,2.0,N,132,68,1.0,2.5,2020-06-17 16:51:01,2020-06-17 17:27:24
3,1.0,1.0,5.20,1.0,N,48,116,2.0,2.5,2020-06-07 14:17:39,2020-06-07 14:33:36
4,2.0,1.0,19.06,1.0,N,77,115,1.0,0.0,2020-06-27 15:23:57,2020-06-27 15:57:13
...,...,...,...,...,...,...,...,...,...,...,...
44945,2.0,1.0,1.08,1.0,N,140,43,1.0,2.5,2020-06-27 20:08:24,2020-06-27 20:15:20
44946,2.0,2.0,1.44,1.0,N,161,143,2.0,2.5,2020-06-26 10:44:25,2020-06-26 10:54:13
44947,1.0,1.0,6.40,1.0,N,161,116,1.0,2.5,2020-06-22 13:03:40,2020-06-22 13:32:33
44948,2.0,3.0,1.97,1.0,N,125,68,1.0,2.5,2020-06-19 18:26:49,2020-06-19 18:34:53


In [1359]:
cat_var = ['VendorID', 'passenger_count','RatecodeID','store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type']

### Transformer to handel datetime and add time_of_ride column

In [1360]:
class DateTimeTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, fmt = '%Y-%m-%d %H:%M:%S'):
        self.fmt = fmt
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        a1 = X['tpep_pickup_datetime']
        a2 = X['tpep_dropoff_datetime']
        a1_d = pd.to_datetime(a1, format = self.fmt)
        a2_d = pd.to_datetime(a2, format = self.fmt)
        X['time_of_ride'] = (a2_d - a1_d).dt.total_seconds()/60
        X = X.drop(columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
        return X

In [1361]:
class LogTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, delta = 1.0):
        self.delta = delta
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        return np.log(X.astype('float')+self.delta)

### Transformer to convert ndarray to Dataframe with given columns

In [1362]:
class ColumnCombiner(TransformerMixin, BaseEstimator):
    def __init__(self, columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime','VendorID', 'passenger_count','RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'trip_distance', 'congestion_surcharge']):
        self.columns = columns
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X,  y = None):
        return pd.DataFrame(X, columns = self.columns)
        

In [1363]:
cat_var = ['VendorID', 'RatecodeID', 'PULocationID', 'DOLocationID', 'payment_type']
cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime'] + cat_var + ['trip_distance', 'congestion_surcharge']

### Imputer Pipeline

In [1364]:
imputer = ColumnTransformer([
    ('datetime', SimpleImputer(strategy = 'constant', fill_value = '2020-06-23 09:30:56'), ['tpep_pickup_datetime', 'tpep_dropoff_datetime']),
    ('mode_imputer', SimpleImputer(strategy = 'most_frequent'), cat_var),
    ('median_imputer', SimpleImputer(strategy = 'median'), ['trip_distance', 'congestion_surcharge'])])

In [1365]:
imputer_pipeline = Pipeline([('imputer', imputer),
                                 ('col_combine', ColumnCombiner(cols)),
                                 ('datetime', DateTimeTransformer())])

### Preprocessing Pipeline

### Since PUloc and DOloc are high level catergorical features, using TargetEncoding

In [1366]:
oh = ['VendorID', 'RatecodeID', 'payment_type']
num = ['trip_distance', 'congestion_surcharge', 'time_of_ride']

In [1367]:
preprocessing_pipeline = ColumnTransformer([('onehot', OneHotEncoder(), oh),
                                           ('num',  RobustScaler(), num),
                                           ('targetencoder', Pipeline([('target', TargetEncoder()),
                                                                       ('scaling', LogTransformer())]), ['PULocationID', 'DOLocationID'])])

In [1368]:
input_pipeline = Pipeline([('imputer', imputer_pipeline),
                           ('prepocess', preprocessing_pipeline)])

In [1369]:
data = input_pipeline.fit_transform(x_train, y_train)

C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


## After fiddeling with different features , the following features were removed as they did not improve the model performance

#### passenger_count
#### store_and_fwd_flag

#### taxis's do not charge on the basis of number of passengers

### Different scaling techniques were applied on the numerical columns, and RobustScaling worked the best
### Despite the skewed nature of numerical features, log transformation ended up worsening the predictions, hence it was not applied

### GridSearchCV returned nan values constantly and hence hyperparameters were manually tuned

## Linear Regression

In [1370]:
lr = Pipeline([('input', input_pipeline),
               ('model', LinearRegression())])

In [1371]:
lr.fit(x_train, y_train)

C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('input',
                 Pipeline(steps=[('imputer',
                                  Pipeline(steps=[('imputer',
                                                   ColumnTransformer(transformers=[('datetime',
                                                                                    SimpleImputer(fill_value='2020-06-23 '
                                                                                                             '09:30:56',
                                                                                                  strategy='constant'),
                                                                                    ['tpep_pickup_datetime',
                                                                                     'tpep_dropoff_datetime']),
                                                                                   ('mode_imputer',
                                                                                    Si

In [1372]:
lr.score(x_val, y_val) # lasso and ridge performed worse than OLS. R^2 in OLS : 0.914

0.9144806353348861

### LinearSVR

In [1373]:
lsvr = Pipeline([('input', input_pipeline),
                 ('model', LinearSVR(epsilon = 1.5, C = 0.9))])

In [1374]:
lsvr.fit(x_train, y_train)

C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('input',
                 Pipeline(steps=[('imputer',
                                  Pipeline(steps=[('imputer',
                                                   ColumnTransformer(transformers=[('datetime',
                                                                                    SimpleImputer(fill_value='2020-06-23 '
                                                                                                             '09:30:56',
                                                                                                  strategy='constant'),
                                                                                    ['tpep_pickup_datetime',
                                                                                     'tpep_dropoff_datetime']),
                                                                                   ('mode_imputer',
                                                                                    Si

In [1375]:
lsvr.score(x_val, y_val) # 0.9123

0.9122799432563371

### Random Forest

In [1376]:
rfr = Pipeline([('input', input_pipeline), 
                ('model', RandomForestRegressor(max_depth = 7))])

In [1377]:
rfr.fit(x_train, y_train)

C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('input',
                 Pipeline(steps=[('imputer',
                                  Pipeline(steps=[('imputer',
                                                   ColumnTransformer(transformers=[('datetime',
                                                                                    SimpleImputer(fill_value='2020-06-23 '
                                                                                                             '09:30:56',
                                                                                                  strategy='constant'),
                                                                                    ['tpep_pickup_datetime',
                                                                                     'tpep_dropoff_datetime']),
                                                                                   ('mode_imputer',
                                                                                    Si

In [1378]:
rfr.score(x_val, y_val) # 0.926

0.9253496142952654

In [1379]:
rfr.score(x_train, y_train) # 0.94

0.9404882544158479

### GradientBoostedTrees

In [1391]:
gbr = Pipeline([('input', input_pipeline), 
                ('model', GradientBoostingRegressor(max_depth = 5))])

In [1392]:
gbr.fit(x_train, y_train)

C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('input',
                 Pipeline(steps=[('imputer',
                                  Pipeline(steps=[('imputer',
                                                   ColumnTransformer(transformers=[('datetime',
                                                                                    SimpleImputer(fill_value='2020-06-23 '
                                                                                                             '09:30:56',
                                                                                                  strategy='constant'),
                                                                                    ['tpep_pickup_datetime',
                                                                                     'tpep_dropoff_datetime']),
                                                                                   ('mode_imputer',
                                                                                    Si

In [1393]:
gbr.score(x_val, y_val) # 0.932

0.9328139642198955

In [1394]:
gbr.score(x_train, y_train) # 0.957

0.957241242146941

### Adaboost

In [1395]:
ab = Pipeline([('input', input_pipeline),
                ('model', AdaBoostRegressor(DecisionTreeRegressor(max_depth = 7)))])

In [1396]:
ab.fit(x_train, y_train)

C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('input',
                 Pipeline(steps=[('imputer',
                                  Pipeline(steps=[('imputer',
                                                   ColumnTransformer(transformers=[('datetime',
                                                                                    SimpleImputer(fill_value='2020-06-23 '
                                                                                                             '09:30:56',
                                                                                                  strategy='constant'),
                                                                                    ['tpep_pickup_datetime',
                                                                                     'tpep_dropoff_datetime']),
                                                                                   ('mode_imputer',
                                                                                    Si

In [1397]:
ab.score(x_val, y_val) # 0.894

0.8901825153814947

In [1398]:
ab.score(x_train, y_train) # 0.92

0.9190681563376577

### VotingRegressor

In [1399]:
vr = VotingRegressor([('lr', clone(lr)), ('lsvr', clone(lsvr)), ('rfr', clone(rfr)), ('gbr', clone(gbr))])

In [1400]:
vr.fit(x_train, y_train)

C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


VotingRegressor(estimators=[('lr',
                             Pipeline(steps=[('input',
                                              Pipeline(steps=[('imputer',
                                                               Pipeline(steps=[('imputer',
                                                                                ColumnTransformer(transformers=[('datetime',
                                                                                                                 SimpleImputer(fill_value='2020-06-23 '
                                                                                                                                          '09:30:56',
                                                                                                                               strategy='constant'),
                                                                                                                 ['tpep_pickup_datetime',
                               

In [1401]:
vr.score(x_val,y_val)

0.9287437020012922

### Neural Network

In [1270]:
def rsquared(true, pred):
    y_bar = np.mean(true)
    ssres = ((true - pred)**2).sum()
    sstot = ((true-y_bar)**2).sum()
    return 1 - ssres/sstot
    
    

In [1234]:
train_data = input_pipeline.fit_transform(x_train, y_train)
val_data = input_pipeline.transform(x_val)

C:\Users\mayur\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [1251]:
model = keras.Sequential([keras.layers.Dense(50, activation = 'relu'),
                          keras.layers.Dense(50, activation = 'relu'),
                          keras.layers.Dense(1)])

In [1252]:
model.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.MeanSquaredError(), metrics = ['mse'])

In [1253]:
es = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
lrs = keras.callbacks.ReduceLROnPlateau(patience = 5, factor = 0.5)

In [1254]:
model.fit(train_data, y_train, validation_data = (val_data, y_val), epochs = 100, callbacks = [lrs, es])

Epoch 1/100
1405/1405 [==============================] - 4s 3ms/step - loss: 27.8915 - mse: 27.8915 - val_loss: 14.6851 - val_mse: 14.6851
Epoch 2/100
1405/1405 [==============================] - 4s 3ms/step - loss: 15.3202 - mse: 15.3202 - val_loss: 13.9027 - val_mse: 13.9027
Epoch 3/100
1405/1405 [==============================] - 4s 3ms/step - loss: 14.2531 - mse: 14.2531 - val_loss: 13.0286 - val_mse: 13.0286
Epoch 4/100
1405/1405 [==============================] - 4s 3ms/step - loss: 13.7069 - mse: 13.7069 - val_loss: 13.0722 - val_mse: 13.0722
Epoch 5/100
1405/1405 [==============================] - 4s 3ms/step - loss: 13.5652 - mse: 13.5652 - val_loss: 12.6979 - val_mse: 12.6979
Epoch 6/100
1405/1405 [==============================] - 4s 3ms/step - loss: 13.0765 - mse: 13.0765 - val_loss: 12.9322 - val_mse: 12.9322
Epoch 7/100
1405/1405 [==============================] - 4s 3ms/step - loss: 13.0484 - mse: 13.0484 - val_loss: 12.1549 - val_mse: 12.1549
Epoch 8/100
1405/1405 [====

1405/1405 [==============================] - 4s 3ms/step - loss: 11.4283 - mse: 11.4283 - val_loss: 11.5843 - val_mse: 11.5843
Epoch 60/100
1405/1405 [==============================] - 4s 3ms/step - loss: 11.4046 - mse: 11.4046 - val_loss: 11.5758 - val_mse: 11.5758
Epoch 61/100
1405/1405 [==============================] - 4s 3ms/step - loss: 11.3998 - mse: 11.3998 - val_loss: 11.5898 - val_mse: 11.5898
Epoch 62/100
1405/1405 [==============================] - 4s 3ms/step - loss: 11.3963 - mse: 11.3963 - val_loss: 11.5770 - val_mse: 11.5770
Epoch 63/100
1405/1405 [==============================] - 4s 3ms/step - loss: 11.3925 - mse: 11.3925 - val_loss: 11.5936 - val_mse: 11.5936
Epoch 64/100
1405/1405 [==============================] - 4s 3ms/step - loss: 11.3923 - mse: 11.3923 - val_loss: 11.5798 - val_mse: 11.5798
Epoch 65/100
1405/1405 [==============================] - 4s 3ms/step - loss: 11.3845 - mse: 11.3845 - val_loss: 11.6568 - val_mse: 11.6568
Epoch 66/100
1405/1405 [=========

In [1259]:
preds = model.predict(val_data)[]

In [1268]:
preds[:, 0]

array([ 8.097335 ,  7.4595976, 33.660072 , ...,  8.1153965,  9.043414 ,
       10.084909 ], dtype=float32)

In [1273]:
rsquared(y_val, preds[:, 0]) # 0.9325

0.9325809022053546

## overall Gradient Boosting Regressor performed the best

In [1274]:
def load_data():
    train = pd.read_csv('train_data_iitm.csv')
    test = pd.read_csv('test_data_iitm.csv')
    tmz = pd.read_csv('taxi_zone_mapping.csv')
    return train, test, tmz

In [1275]:
train, test, tmz = load_data()

In [1402]:
testing_data = test[x_train.columns]

In [1404]:
test_preds = gbr.predict(testing_data)

In [1407]:
test['fare_amount'] = test_preds

In [1408]:
test

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,congestion_surcharge,fare_amount
0,2.0,2020-06-30 18:39:30,2020-06-30 18:45:11,1.0,1.66,1.0,N,48,90,1.0,2.5,7.295117
1,2.0,2020-06-27 23:16:17,2020-06-27 23:23:17,3.0,1.50,1.0,N,48,246,2.0,2.5,7.471939
2,1.0,2020-06-09 12:54:15,2020-06-09 13:01:25,1.0,1.90,1.0,N,237,234,1.0,2.5,7.970325
3,2.0,2020-06-20 09:42:39,2020-06-20 09:45:12,1.0,0.27,1.0,N,170,170,2.0,2.5,3.912693
4,2.0,2020-06-23 18:31:46,2020-06-23 18:48:34,2.0,4.06,1.0,N,186,238,2.0,2.5,15.301970
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,2020-06-09 23:25:53,2020-06-09 23:26:01,1.0,0.00,1.0,N,74,74,3.0,0.0,0.445621
9996,2.0,2020-06-03 13:07:44,2020-06-03 13:11:27,1.0,0.54,1.0,N,141,237,1.0,2.5,4.615164
9997,2.0,2020-06-07 18:40:08,2020-06-07 18:46:23,1.0,1.14,1.0,N,41,74,1.0,0.0,7.164447
9998,2.0,2020-06-12 15:27:56,2020-06-12 15:42:42,1.0,2.92,1.0,N,88,65,1.0,2.5,13.077529


In [1409]:
test.to_csv('test_data_iitm_solved.csv', index = False)